In [ ]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket_tmp/crocket')

from datetime import datetime
import itertools
from math import floor
from numpy import exp, linspace, median, mean, std
from pprint import pprint
from os import environ
from os.path import join
from scipy.stats import mannwhitneyu, ttest_ind, variation


import matplotlib.pyplot as plt
%matplotlib inline

from crocket.sql.sql import Database
from crocket.utilities.credentials import get_credentials

In [ ]:
# Input parameters

duration = 3

In [ ]:
# Environment variables

HOME_DIRECTORY_PATH = environ['HOME']
CREDENTIALS_FILE_PATH = join(HOME_DIRECTORY_PATH, '.credentials_unlocked.json')

HOSTNAME = 'localhost'
DATABASE_NAME = 'BITTREX3'

In [ ]:
# Load SQL username and password
USERNAME, PASSCODE = get_credentials(CREDENTIALS_FILE_PATH)

# Create database connection
db = Database(hostname=HOSTNAME,
              username=USERNAME,
              password=PASSCODE,
              database_name=DATABASE_NAME)

In [ ]:
tables = db.get_all_tables()

In [ ]:
def call(time, wprice, buyvolume, buyorder, 
         duration=None, 
         price_lag_time=None, 
         price_lag_duration=None,
         price_lag_threshold=None,
         volume_lag_duration=None,
         volume_lag_threshold=None,
         profit_percent=None, 
         stop_loss_percent=None, 
         stop_gain_percent=None, 
         max_hold_time=None, 
         wait_time=None):
    
    calls = []
    
    bought = False
    stop_gain = False
    maximize_gain = False
    
    for ii in range(duration+volume_lag_duration, len(buyvolume)):
        
        # Action if haven't bought coin
        if not bought:
            
            # No action if purchased within time of last buy
            try:
                if ii - calls[-1].get('start') < wait_time:
                    continue
            except IndexError:
                pass
                
            sample_volume_mean = mean(buyvolume[ii-duration:ii])
            volume_lag_median = median(buyvolume[ii-duration-volume_lag_duration:ii-duration])
            
            if sample_volume_mean > 0 and volume_lag_median < volume_lag_threshold:
                sample_price = float(wprice[ii-1])
                previous_price = float(mean(wprice[ii-duration-price_lag_time:ii-duration-price_lag_time+price_lag_duration]))

                if sample_volume_mean > 2 and \
                abs((sample_price - previous_price)/previous_price) < price_lag_threshold and \
                sum([1 if x > 1 else 0 for x in buyvolume[ii-duration:ii]]) >= 3:
                    
                    bought = True
                    calls.append({'start': ii,  # buy time offset by 1 minute, account for time to purchase
                                  'time': time[ii],
                                  'buy_price': float(wprice[ii]),
                                  'buy_volume': sample_volume_mean,
                                  'buy_volume_total': sum(buyvolume[ii-duration:ii])}) 

        # Action if have bought coin
        else:
            
            # Set stop loss after X minutes after buy
            if ii - calls[-1].get('start') > 0 and \
            wprice[ii] < (calls[-1].get('buy_price') * (1 - stop_loss_percent)):
                
                print(ii, 'Passed stop loss.')
                calls[-1]['stop'] = ii + 1 # sell price offset by 1 minute, account for time to sell
                calls[-1]['sell_price'] = float(wprice[ii+1])
                calls[-1]['profit'] = ((calls[-1].get('sell_price') - calls[-1].get('buy_price'))/calls[-1].get('buy_price')) * 0.9975**2
                
                bought = False
                stop_gain = False
                
                continue
            
            # Activate stop gain signal after passing threshold percentage
            if wprice[ii] > (calls[-1].get('buy_price') * (stop_gain_percent + 1)):
                stop_gain = True
            
            # Activate maximize gain signal after passing profit threshold
            if wprice[ii] > (calls[-1].get('buy_price') * (profit_percent + 1)):
                maximize_gain = True
            
            # Sell after hitting profit threshold followed by drop in price of X%
            if maximize_gain:
#                 if (wprice[ii] - wprice[ii-1])/wprice[ii-1] < -0.005 or \
#                 (wprice[ii] < wprice[ii-1] < wprice[ii-2]) or \
#                 wprice[ii] < (calls[-1].get('buy_price') * (profit_percent + 1)):
                calls[-1]['stop'] = ii + 1 # sell price offset by 1 minute, account for time to sell
                calls[-1]['sell_price'] = float(wprice[ii+1])
                calls[-1]['profit'] = ((calls[-1].get('sell_price') - calls[-1].get('buy_price'))/calls[-1].get('buy_price')) * 0.9975**2

                bought = False
                stop_gain = False
                maximize_gain = False

                continue
                
            # Sell after passing max hold time
            if not maximize_gain and \
            (ii - calls[-1].get('start')) > max_hold_time:
                

                calls[-1]['stop'] = ii + 1 # sell price offset by 1 minute, account for time to sell
                calls[-1]['sell_price'] = float(wprice[ii+1])
                calls[-1]['profit'] = ((calls[-1].get('sell_price') - calls[-1].get('buy_price'))/calls[-1].get('buy_price')) * 0.9975**2
                
                print('Passed target hold time. Time: {}, Buy: {}, Sell: {}, Profit: {}'.format(
                    str(calls[-1].get('time')), str(calls[-1].get('buy_price')), 
                    str(calls[-1].get('sell_price')), str(calls[-1].get('profit'))))
                
                bought = False
                stop_gain = False
            
            # Sell after detecting stop gain signal and price drop below stop gain price
            elif stop_gain and wprice[ii] < (calls[-1].get('buy_price') * (stop_gain_percent + 1)):
                
                #print(ii, 'Passed stop gain then below stop gain, exited.')
                calls[-1]['stop'] = ii + 1 # sell price offset by 1 minute, account for time to sell
                calls[-1]['sell_price'] = float(wprice[ii+1])
                calls[-1]['profit'] = ((calls[-1].get('sell_price') - calls[-1].get('buy_price'))/calls[-1].get('buy_price')) * 0.9975**2

                bought = False
                stop_gain = False
                maximize_gain = False
    
    return calls

In [ ]:
def benchmark(skip_list, params):
    
    tables = db.get_all_tables()
    
    all_calls = {}
    
    total_profit = 0
    
    for table in tables:
        
        if table not in skip_list:
            data = db.select_query(table, '*', 'where time < "2017/11/24 20:14:00"')
            
            if data:
                time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)

                calls = call(time, wprice, buyvolume, buyorder,
                             duration=params.get('duration'),
                             price_lag_time=params.get('price_lag_time'),
                             price_lag_duration=params.get('price_lag_duration'),
                             price_lag_threshold=params.get('price_lag_threshold'),
                             volume_lag_duration=params.get('volume_lag_duration'),
                             volume_lag_threshold=params.get('volume_lag_threshold'),
                             profit_percent=params.get('profit_percent'),
                             stop_loss_percent=params.get('stop_loss_percent'),
                             stop_gain_percent=params.get('stop_gain_percent'),
                             max_hold_time=params.get('max_hold_time'),
                             wait_time=params.get('wait_time'))

                if calls:
                    all_calls[table] = calls
                    profit = sum([x.get('profit') for x in calls if 'profit' in x])
                    print('For {}, profit: {}'.format(table, str(profit)))

                    total_profit += profit
    
    print('Total profit: {}'.format(total_profit))
    
    return all_calls

In [ ]:
skip_list = ['BTC-BCC', 'BTC-ETH', 'BTC-LSK', 'BTC-NEO', 'BTC-XRP', 'BTC-LTC']

params = {'duration': 3,
          'price_lag_time': 30,
          'price_lag_duration': 5,
          'price_lag_threshold': 0.03,
          'volume_lag_duration': 30,
          'volume_lag_threshold': 2,
          'profit_percent': 0.05,
          'stop_loss_percent': 0.01,
          'stop_gain_percent': 0.02,
          'max_hold_time': 180,
          'wait_time': 240}

calls = benchmark(skip_list, params)

In [ ]:
skip_list = ['BTC-BCC', 'BTC-ETH', 'BTC-LSK', 'BTC-NEO', 'BTC-XRP', 'BTC-LTC']

params = {'duration': 3,
          'price_lag_time': 30,
          'price_lag_duration': 5,
          'price_lag_threshold': 0.03,
          'volume_lag_duration': 30,
          'volume_lag_threshold': 2,
          'profit_percent': 0.05,
          'stop_loss_percent': 0.01,
          'stop_gain_percent': 0.02,
          'max_hold_time': 240,
          'wait_time': 240}

calls = benchmark(skip_list, params)

In [ ]:
len(list(itertools.chain.from_iterable([calls.get(k) for k in calls])))

In [ ]:
list(itertools.chain.from_iterable([[calls.get(k)] for k in calls]))

In [ ]:
for k in sorted(calls.keys()):

    print('{}: total profit: {}, loss: {}, calls: {}, calls @ profit threshold: {}'.format(
        k, str(sum([x.get('profit') for x in calls[k]])), str(sum([x.get('profit') for x in calls[k] if x.get('profit') < 0])), 
        len(calls[k]), str(len([x for x in calls[k] if x.get('profit') > 0.05]))))

In [ ]:
mv = {'loss': [],
      'gain': []}

tv = {'loss': [],
      'gain': []}

tp = [0 for x in range(50)]

for k in sorted(calls.keys()):
    
    for e in calls.get(k):
        
        if e.get('profit') <= 0.02:
            mv['loss'].append(float(e.get('buy_volume')))
            tv['loss'].append(float(e.get('buy_volume_total')))
        else:
            mv['gain'].append(float(e.get('buy_volume')))
            tv['gain'].append(float(e.get('buy_volume_total')))
            
        tmp = floor(float(e.get('buy_volume_total')))

        tp[tmp] += float(e.get('profit'))
            
        print('{}: profit: {}, mean_volume: {}, total_volume: {}'.format(
            k, str(e.get('profit')), str(e.get('buy_volume')), str(e.get('buy_volume_total'))))

In [ ]:
calls.get('BTC-TRIG')

In [ ]:
for ii in range(17, 25):

    orders = list(itertools.chain.from_iterable([[x for x in calls.get(y) if 0 < (x.get('time')-datetime(2017, 11, ii, 20, 14)).total_seconds() < 86400] 
                  for y in calls]))
    
    print('11/{}: {} orders, profit: {}'.format(str(ii), str(len(orders)), str(sum([x.get('profit') for x in orders]))))
    
    pprint(sorted([(x.get('time'), x.get('profit')) for x in orders]))

In [ ]:
for x in sorted(calls.keys()):
    
    pprint((x, calls.get(x)))

In [ ]:
data = db.select_query('BTC-BAY', '*')

time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)

calls = call(time, wprice, buyvolume, buyorder,
                             duration=params.get('duration'),
                             price_lag_time=params.get('price_lag_time'),
                             price_lag_duration=params.get('price_lag_duration'),
                             price_lag_threshold=params.get('price_lag_threshold'),
                             volume_lag_duration=params.get('volume_lag_duration'),
                             volume_lag_threshold=params.get('volume_lag_threshold'),
                             profit_percent=params.get('profit_percent'),
                             stop_loss_percent=params.get('stop_loss_percent'),
                             stop_gain_percent=params.get('stop_gain_percent'),
                             max_hold_time=params.get('max_hold_time'),
                             wait_time=params.get('wait_time'))

In [ ]:
calls

In [ ]:
for table in tables:
    
    data = db.select_query(table, '*')

    time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)
    
    ind = [i for i in range(duration, len(buyvolume)) if mean(buyvolume[i-duration:i]) > 2 
           and variation(buyvolume[i-duration:i]) < 1
           and all([1 if x > 1 else 0 for x in buyvolume[i-duration:i]])]
    
    if ind:
        print('Found {} hits for {}: {}'.format(str(len(ind)), table, ','.join(map(str, ind))))
        plt.plot(range(len(time)), wprice, markevery=ind, marker='o', markerfacecolor='r', markersize=10)
        plt.show()
        plt.plot(range(len(time)), buyvolume, markevery=ind, marker='o', markerfacecolor='r', markersize=10)
        plt.show()
        for i in ind:
            print(i, buyvolume[i-duration:i])
            print(i, time[i])
        input('NEXT')
    else:
        print('No hits found for {}'.format(table))

In [ ]:
data = db.select_query('BTC-ADA', '*')

time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)

In [ ]:
start = 1100
stop = 1500
marked = [1270]
plt.plot(time[start:stop], wprice[start:stop], markevery=[x - start for x in marked], marker='o', markerfacecolor='r', markersize=10)
plt.show()

In [ ]:
variation([1, 9, 40])

In [ ]:
m = mean(buyvolume[1273-3:1273])
s = std(buyvolume[1273-3:1273])

In [ ]:
m, s

In [ ]:
[(x, y) for x, y in zip(wprice[1260:1280], buyvolume[1260:1280])]

In [ ]:
buyvolume_ss = [median(buyvolume[x:x+60]) for x in range(len(buyvolume)-60)]

In [ ]:
plt.plot(range(len(buyvolume_ss)), buyvolume_ss)
plt.show()

In [ ]:
plt.plot(range(len(price)), price)
plt.plot(range(len(price)), wprice)
plt.show()

In [ ]:
[(x, y, z) for x, y, z in zip(buyvolume[1700:2000], wprice[1700:2000], buyorder[1700:2000])]

In [ ]:
[(x, y) for x, y in zip(buyvolume[200:300], wprice[200:300])]

In [ ]:
median(buyvolume[241:245])
pprint(buyvolume[241:247])
pprint(price[241:290])

In [ ]:
plt.plot(range(len(time)), wprice)
plt.show()

In [ ]:
plt.plot(range(len(time)), buyvolume)
plt.show()

In [ ]:
# Get data from SQL database

# Get initial coine data

# Get data every minute